In [8]:
import dotenv
from openai import AzureOpenAI
import os
import pandas as pd

dotenv.load_dotenv()
client = AzureOpenAI(
    azure_endpoint= os.environ["AZURE_OPENAI_ENDPOINT"],
    api_key= os.environ["AZURE_OPENAI_API_KEY"],
    api_version = '2024-10-01-preview'
)

deployment = os.environ['AZURE_OPENAI_DEPLOYMENT']

In [9]:
def split_text(text, max_length, min_length):
    words = text.split()
    chunks = []
    current_chunk = []

    for word in words:
        current_chunk.append(word)
        if len(' '.join(current_chunk)) < max_length and len(' '.join(current_chunk)) > min_length:
            chunks.append(' '.join(current_chunk))
            current_chunk = []
    if current_chunk:
        chunks.append(' '.join(current_chunk))

    return chunks

In [10]:
file_path = 'rag_file.txt'

with open(file_path, 'r') as file:
    file_content = file.read()
text_list = split_text(file_content,1200,800)

In [11]:
embeddings = []
for items in text_list:

    response = client.embeddings.create(
        input = items,
        model= "text-embedding-ada-002"
    )
    embeddings.append(response.data[0].embedding)

In [14]:

rag_df = pd.DataFrame({
    "chunks":text_list,
    "embedding":embeddings
})

In [15]:
from sklearn.neighbors import NearestNeighbors

embeddings = rag_df['embedding'].to_list()
nbrs = NearestNeighbors(n_neighbors=5, algorithm='ball_tree').fit(embeddings)
distances, indices = nbrs.kneighbors(embeddings)

In [21]:
user_input = "What all changes can be brought in the education system in the country?"

def chatbot(user_input):
    response = client.embeddings.create(
        input = user_input,
        model= "text-embedding-ada-002"
    )
    query_vector = response.data[0].embedding

    distances, indices = nbrs.kneighbors([query_vector])

    history = []
    for index in indices[0]:
        history.append(rag_df['chunks'].iloc[index])

    history.append(user_input)

    messages=[
        {"role": "system", "content": "You are an AI assistant that helps with AI questions."},
        {"role": "user", "content": " ".join(history)}
    ]

    response = client.chat.completions.create(
        model="gpt-35-turbo-16k",
        temperature=0.7,
        max_tokens=800,
        messages=messages
    )

    return response.choices[0].message

chatbot(user_input)

ChatCompletionMessage(content="There are several changes that can be brought into the education system in India to address the challenges and improve its overall effectiveness. Here are some key changes that can be considered:\n\n1. Emphasis on skill development: The education system should focus on developing practical skills alongside academic knowledge. This can be achieved through vocational training programs, internships, and apprenticeships that provide students with real-world experience and prepare them for the job market.\n\n2. Integration of technology: Technology should be integrated into classrooms to enhance teaching and learning processes. This includes providing access to computers, internet connectivity, and digital learning resources. Teachers should also be trained to effectively use technology in their teaching methods.\n\n3. Modernizing curriculum: The curriculum should be updated to include relevant and contemporary subjects, such as coding, artificial intelligence